In [25]:
import pandas as pd
from scipy.signal import savgol_filter
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import numpy as np


def preprocessing_v1():
    
    train_data_og = pd.read_csv('/Users/maelysclerget/Desktop/ML/bio322_project/epfl-bio-322-2024/train.csv')
    test_data_og = pd.read_csv('/Users/maelysclerget/Desktop/ML/bio322_project/epfl-bio-322-2024/test.csv')
    train_data = train_data_og.copy()
    test_data = test_data_og.copy()
    train_data = train_data.drop(columns=['prod_substance'])
    test_data = test_data.drop(columns=['prod_substance'])
    
    non_wavelength_cols = ['device_serial','substance_form_display','measure_type_display']
    wavelength_cols = train_data.columns[6:]
    
    #One Hot encoding 
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    X_train_encoded = encoder.fit_transform(train_data[non_wavelength_cols])
    X_test_encoded = encoder.transform(test_data[non_wavelength_cols])
    
    # Convert encoded features to DataFrame
    X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(non_wavelength_cols))
    X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(non_wavelength_cols))
    
    train_data_combined = pd.concat([pd.DataFrame(X_train_encoded_df), train_data[wavelength_cols].reset_index(drop=True)], axis=1)
    test_data_combined = pd.concat([pd.DataFrame(X_test_encoded_df), test_data[wavelength_cols].reset_index(drop=True)], axis=1)
    
    # Add sample_name column back to the combined DataFrames
    train_data_combined.insert(0, 'sample_name', train_data_og['sample_name'])
    test_data_combined.insert(0, 'sample_name', test_data_og['sample_name'])
    
    #Remove NaN values
    train_data_combined = train_data_combined.dropna()
    test_data_combined = test_data_combined.dropna()
    
    y_train = train_data['PURITY'].iloc[train_data_combined.index]
    
    #Standardize the data
    scaler = StandardScaler()
    wavelength_train_scaled = scaler.fit_transform(train_data_combined[wavelength_cols])
    wavelength_test_scaled = scaler.transform(test_data_combined[wavelength_cols])
    
    train_data_combined[wavelength_cols] = wavelength_train_scaled
    test_data_combined[wavelength_cols] = wavelength_test_scaled
    
    # Apply Savitzky-Golay filter
    #window_length = 7  # Choose an appropriate window length
    #polyorder = 3      # Choose an appropriate polynomial order
    #train_data_combined[wavelength_cols] = savgol_filter(train_data_combined[wavelength_cols], window_length, polyorder, axis=0)
    #test_data_combined[wavelength_cols] = savgol_filter(test_data_combined[wavelength_cols], window_length, polyorder, axis=0)
    
    #outliers
    outliers_index = (np.abs(wavelength_train_scaled) > 3).any(axis=1)
    train_data_combined = train_data_combined[~outliers_index] #exclude outliers
    y_train = y_train[~outliers_index]
    
    X_train_final = train_data_combined.reset_index(drop=True)
    X_test_final = test_data_combined.reset_index(drop=True)
    Y_train_final = y_train.reset_index(drop=True)
    
    X_train_final.to_csv('/Users/maelysclerget/Desktop/ML/bio322_project/epfl-bio-322-2024/preprocessed_train_data.csv', index=False)
    X_test_final.to_csv('/Users/maelysclerget/Desktop/ML/bio322_project/epfl-bio-322-2024/preprocessed_test_data.csv', index=False)
    
    return X_train_final, X_test_final, Y_train_final

def correlation():
    
    X_train, X_test, y_train = preprocessing_v1()
     
    wavelength_cols = X_train.columns[54:]
    
    # Compute correlation matrix only for wavelength columns
    correlation_matrix = X_train[wavelength_cols].corr()

    # Visualize correlation matrix
    import seaborn as sns
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False)
    plt.title("Correlation Matrix for Wavelength Features")
    plt.show()

    # Identify highly correlated features (e.g., |r| > 0.95)
    threshold_high = 0.999
    threshold_low = 0.2

    high_corr_pairs = [
        (i, j)
        for i in range(correlation_matrix.shape[0])
        for j in range(i + 1, correlation_matrix.shape[1])
        if abs(correlation_matrix.iloc[i, j]) > threshold_high
    ]
    
    features_to_drop = set()
    for i, j in high_corr_pairs:
        features_to_drop.add(wavelength_cols[j])  # Arbitrarily drop the second feature in the pair

    # Remove the selected features
    X_train_reduced = X_train.drop(columns=list(features_to_drop))
    X_test_reduced = X_test.drop(columns=list(features_to_drop))
    
    low_corr_pairs = [
        (i, j)
        for i in range(correlation_matrix.shape[0])
        for j in range(i + 1, correlation_matrix.shape[1])
        if abs(correlation_matrix.iloc[i, j]) < threshold_low
    ]

    print("Highly correlated features:")
    for i, j in high_corr_pairs:
        print(f"{wavelength_cols[i]} and {wavelength_cols[j]}: {correlation_matrix.iloc[i, j]}")
    print("Low correlated features:")
    for i, j in low_corr_pairs:
        print(f"{wavelength_cols[i]} and {wavelength_cols[j]}: {correlation_matrix.iloc[i, j]}")
        
    return X_train_reduced, X_test_reduced

def linear_regression_drop_features():
    
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score
    
    X_train, X_test = correlation()
    _, _, y_train = preprocessing_v1()
    X_train = X_train.drop(columns=['sample_name'])
    X_test = X_test.drop(columns=['sample_name'])
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    mse = mean_squared_error(y_train, y_train_pred)
    print('Training MSE:', mse)
    
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    print('CV MSE:', -cv_scores.mean())
    
    # Predict on test data
    y_test_pred = model.predict(X_test)
    
    # Create submission DataFrame
    submission = submission_file(y_test_pred)
    
    # Save submission to CSV
    submission.to_csv('/Users/maelysclerget/Desktop/ML/bio322_project/epfl-bio-322-2024/sample_submission_LR_drop_features.csv', index=False)
    print('Submission file saved successfully.')   

def pca():
    
    from sklearn.decomposition import PCA
    import numpy as np
    
    X_train, X_test, y_train = preprocessing_v1()
    wavelength_cols = X_train.columns[54:]

    # Perform PCA on scaled wavelength columns
    pca = PCA(n_components=5)
    X_train_pca = pca.fit_transform(X_train[wavelength_cols])
    X_test_pca = pca.transform(X_test[wavelength_cols])

    # # Analyze cumulative variance explained
    # explained_variance = np.cumsum(pca.explained_variance_ratio_)

    # # Plot cumulative variance explained
    # plt.figure(figsize=(8, 6))
    # plt.plot(range(1, len(explained_variance) + 1), explained_variance, marker='o', linestyle='--')
    # plt.axhline(y=0.999, color='r', linestyle='-')
    # plt.title("Cumulative Explained Variance")
    # plt.xlabel("Number of Principal Components")
    # plt.ylabel("Cumulative Variance Explained")
    # plt.grid()
    # plt.show()

    # # Number of components explaining >95% variance
    # n_components_999 = np.argmax(explained_variance >= 0.999) + 1
    # print(f"Number of components explaining >99.9% variance: {n_components_999}")

    X_train_combined = pd.concat([X_train.iloc[:, :54].reset_index(drop=True), 
                                  pd.DataFrame(X_train_pca, columns=[f'PC{i+1}' for i in range(5)])], axis=1)
    X_test_combined = pd.concat([X_test.iloc[:, :54].reset_index(drop=True), 
                                 pd.DataFrame(X_test_pca, columns=[f'PC{i+1}' for i in range(5)])], axis=1)
        
    X_train_combined.to_csv('/Users/maelysclerget/Desktop/ML/bio322_project/epfl-bio-322-2024/PCA_train.csv', index=False)
    X_test_combined.to_csv('/Users/maelysclerget/Desktop/ML/bio322_project/epfl-bio-322-2024/PCA_test.csv', index=False)

    print('PCA files saved successfully.')
    
    return X_train_combined, X_test_combined

def linear_regression():
    
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score
    
    X_train, X_test, y_train = preprocessing_v1()
    X_train = X_train.drop(columns=['sample_name'])
    X_test = X_test.drop(columns=['sample_name'])
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Predict on training data
    y_train_pred = model.predict(X_train)
    mse = mean_squared_error(y_train, y_train_pred)
    print('Training MSE:', mse)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    print('CV MSE:', -cv_scores.mean())
    
    # Predict on test data
    y_test_pred = model.predict(X_test)
    
    # Create submission DataFrame
    submission = submission_file(y_test_pred)
    
    # Save submission to CSV
    submission.to_csv('/Users/maelysclerget/Desktop/ML/bio322_project/epfl-bio-322-2024/sample_submission_LR.csv', index=False)
    print('Submission file saved successfully.')
    
    # Print feature importance
    feature_importance = pd.Series(model.coef_, index=X_train.columns)
    feature_importance = feature_importance.abs().sort_values(ascending=False)
    wavelength_feature_importance_df = feature_importance.reset_index()
    wavelength_feature_importance_df.columns = ['Feature', 'Importance']
    wavelength_feature_importance_df.to_csv('/Users/maelysclerget/Desktop/ML/bio322_project/epfl-bio-322-2024/feature_importance_LR1.csv', index=False)
    print('Feature Importance saved successfully.')
    
    # Calculate stats threshold
    threshold1 = feature_importance.quantile(0.25)
    threshold2 = feature_importance.mean()
    threshold3 = feature_importance.median()
    
    # Identify low-importance features
    low_importance_features = feature_importance[feature_importance < threshold1].index
    print(f'Low importance features: {low_importance_features}')
    
    # Remove low-importance features and retrain the model
    X_train_reduced = X_train.drop(columns=low_importance_features)
    X_test_reduced = X_test.drop(columns=low_importance_features)
    
    model_reduced = LinearRegression()
    model_reduced.fit(X_train_reduced, y_train)
    
    # Predict on training data with reduced features
    y_train_pred_reduced = model_reduced.predict(X_train_reduced)
    mse_reduced = mean_squared_error(y_train, y_train_pred_reduced)
    print('Training MSE with reduced features:', mse_reduced)
    
    # Cross-validation with reduced features
    cv_scores_reduced = cross_val_score(model_reduced, X_train_reduced, y_train, cv=5, scoring='neg_mean_squared_error')
    print('CV MSE with reduced features:', -cv_scores_reduced.mean())
    
    # Predict on test data with reduced features
    y_test_pred_reduced = model_reduced.predict(X_test_reduced)
    
    # Create submission DataFrame with reduced features
    submission_reduced = submission_file(y_test_pred_reduced)
    
    # Save submission with reduced features to CSV
    submission_reduced.to_csv('/Users/maelysclerget/Desktop/ML/bio322_project/epfl-bio-322-2024/sample_submission_LR_reduced.csv', index=False)
    print('Submission file with reduced features saved successfully.')
    
def linear_regression_pca():
    
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score

    # Load PCA-transformed training data
    X_train_pca, X_test_pca = pca()
    
    # Load the original test data from preprocessing
    X_train, X_test, y_train = preprocessing_v1()
    X_test_pca = X_test_pca.drop(columns=['sample_name'])  # Drop sample_name for modeling
    X_train_pca = X_train_pca.drop(columns=['sample_name'])
    
    model = LinearRegression()
    model.fit(X_train_pca, y_train)

    # Predict on the training data
    y_train_pred = model.predict(X_train_pca)
    mse_train = mean_squared_error(y_train, y_train_pred)
    print(f'Training MSE: {mse_train}')

    # Cross-validation for validation error
    cv_scores = cross_val_score(model, X_train_pca, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_mse = -cv_scores.mean()
    print(f'Cross-Validation MSE: {cv_mse}')

    # Predict on test data
    y_test_pred = model.predict(X_test_pca)

    # Format test predictions for submission
    submission = submission_file(y_test_pred)
    
    # Save predictions to CSV
    submission.to_csv('/Users/maelysclerget/Desktop/ML/bio322_project/epfl-bio-322-2024/sample_submission_LR_PCA.csv', index=False)
    print('Submission file saved successfully.')

# def polynomial_regression():
    
# def knn():

# PURITY log transforom
    
def submission_file(y_test_predicted):
    submission_reduced = pd.DataFrame({
        'ID': range(1, len(y_test_predicted) + 1),
        'PURITY': y_test_predicted
    })
    return submission_reduced
    
def main():
    #linear_regression()
    #correlation()
    #pca()
    linear_regression_pca()
    #linear_regression_drop_features()
    
if __name__ == '__main__':
    main()


PCA files saved successfully.
Training MSE: 55.5481669002354
Cross-Validation MSE: 1.2504677040306736e+22
Submission file saved successfully.
